In [1]:
import findspark
findspark.init() 

In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.ml.stat import Correlation
from pyspark.sql.types import IntegerType,BooleanType,DateType,FloatType, StringType
import matplotlib.pyplot as plt 
import seaborn as sns
import pyspark.sql.types as T


In [4]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

def fudf(val):
    return functools.reduce(lambda x, y:x+y, val)

In [8]:
df = spark.read.json('C:\\Users\\pasqu\\Desktop\\Università\\Distributed Data Analysis and Mining\\cleaned_dataset')

In [9]:
df.toPandas()

,acousticness,danceability,duration_ms,energy,genres,id_track,instrumentalness,key,liveness,loudness,mode,popularity_track,release_date,speechiness,sum_artist_followers,sum_artist_popularity,tempo,time_signature,valence
0,0.658000,0.602,156067,0.552,"[classic czech pop, czech folk, czech rock]",00AeAaSNbe92PRrstQskvH,0.000000,0,0.0972,-6.667,1,3,1980-01-01,0.4040,10807,80,182.229,3,0.650
1,0.543000,0.770,220133,0.891,"[afrobeat, afropop, world]",00DJt4PjkzeXhKKVDekw2n,0.000796,1,0.0684,-7.306,1,9,1976-01-01,0.1720,19833,43,135.573,4,0.898
2,0.000048,0.212,250960,0.986,"[alternative metal, gothenburg metal, melodic ...",00HgVIkZrAL8WjAN9Et6WW,0.918000,0,0.3240,-6.690,0,33,1996-02-20,0.1400,874600,68,140.917,4,0.231
3,0.144000,0.362,457040,0.453,"[corrosion, dark wave, ethereal wave, gothabil...",00Lx2f8NRiFKMGWa0GBGEt,0.827000,11,0.1170,-17.744,0,35,1990-01-01,0.0398,69129,42,118.853,4,0.257
4,0.957000,0.343,282891,0.225,"[brazilian rock, mpb, pop rock brasileiro, roc...",00fzPML4lrNag28OPzsB93,0.000249,1,0.6610,-14.937,0,52,2017-09-22,0.0384,1709414,68,144.533,4,0.101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583358,0.158000,0.775,198549,0.694,[],7zQEJoYjxZBKUfzGO57isE,0.000000,1,0.1000,-8.266,1,37,2019-04-30,0.1920,0,0,88.022,4,0.866
583359,0.757000,0.544,213787,0.844,[hungarian pop],7zSqJS5YBEzpeLxqR6WNnm,0.000005,10,0.2020,-3.735,1,28,2012-11-12,0.0967,4286,24,79.836,4,0.620
583360,0.001150,0.580,279786,0.877,"[chilean indie, chilean rock, latin alternativ...",7zZESEhPkn3qWqfsgcdiQT,0.024200,7,0.6220,-9.005,1,46,2001-10-08,0.0454,200001,56,89.983,4,0.671
583361,0.000092,0.621,421907,0.544,"[alternative rock, big beat, britpop, dance ro...",7zxDWJtcsVvXsbMBxxbSba,0.579000,8,0.8890,-14.754,1,57,1991-09-23,0.0514,362972,60,91.147,4,0.885


# Data Transformation

droppo le colonne inutili ai fini del clustering e la colonna energy per l'alta correlazione con loudness (che è più correlata a popularity track)

In [26]:
df=df.drop('id_track','popularity_track','sum_artist_followers','sum_artist_popularity','energy')

In [53]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["duration_ms", "danceability", "loudness", "speechiness","acousticness","instrumentalness","liveness","valence","tempo","time_signature"],
    outputCol="features")

df=output = assembler.transform(df)
print("Assembled columns")
output.select("features").show(truncate=False)

Assembled columns
+---------------------------------------------------------------------+
|features                                                             |
+---------------------------------------------------------------------+
|[156067.0,0.602,-6.667,0.404,0.658,0.0,0.0972,0.65,182.229,3.0]      |
|[220133.0,0.77,-7.306,0.172,0.543,7.96E-4,0.0684,0.898,135.573,4.0]  |
|[250960.0,0.212,-6.69,0.14,4.8E-5,0.918,0.324,0.231,140.917,4.0]     |
|[457040.0,0.362,-17.744,0.0398,0.144,0.827,0.117,0.257,118.853,4.0]  |
|[282891.0,0.343,-14.937,0.0384,0.957,2.49E-4,0.661,0.101,144.533,4.0]|
|[217000.0,0.893,-5.331,0.0362,0.119,9.76E-4,0.0952,0.958,120.021,4.0]|
|[545107.0,0.351,-17.481,0.0413,0.96,0.0339,0.083,0.237,107.86,4.0]   |
|[256933.0,0.422,-3.806,0.0799,0.168,0.0,0.129,0.616,97.573,4.0]      |
|[249133.0,0.627,-5.604,0.0694,0.775,0.0,0.0886,0.891,152.277,4.0]    |
|[189800.0,0.554,-14.1,0.0438,0.971,0.0,0.142,0.549,103.292,1.0]      |
|[181733.0,0.189,-14.585,0.0359,0.945,0.831,0.

variance treshold per eliminare gli attributi con alta collinearità (a quanto si setta la treshold?)

In [54]:
from pyspark.ml.feature import VarianceThresholdSelector


selector = VarianceThresholdSelector(varianceThreshold=8.0, outputCol="selectedFeatures")

result = selector.fit(df).transform(df)

print("Output: Features with variance lower than %f are removed." %
      selector.getVarianceThreshold())
result.show()

Output: Features with variance lower than 8.000000 are removed.
+------------+------------+-----------+--------------------+----------------+---+--------+--------+----+------------+-----------+-------+--------------+-------+--------------------+--------------------+
|acousticness|danceability|duration_ms|              genres|instrumentalness|key|liveness|loudness|mode|release_date|speechiness|  tempo|time_signature|valence|            features|    selectedFeatures|
+------------+------------+-----------+--------------------+----------------+---+--------+--------+----+------------+-----------+-------+--------------+-------+--------------------+--------------------+
|       0.658|       0.602|     156067|[classic czech po...|             0.0|  0|  0.0972|  -6.667|   1|  1980-01-01|      0.404|182.229|             3|   0.65|[156067.0,0.602,-...|[156067.0,-6.667,...|
|       0.543|        0.77|     220133|[afrobeat, afropo...|         7.96E-4|  1|  0.0684|  -7.306|   1|  1976-01-01|      0

PCA sugli attributi continui per trovare nuovi attributi che riassumano (catturando la varianza) quelli presenti


In [55]:
from pyspark.ml.feature import PCA


pca = PCA(k=3, inputCol="features", outputCol="pcaFeatures")
model = pca.fit(df)

result = model.transform(df).select("pcaFeatures")
result.show(truncate=False)

+-----------------------------------------------------------+
|pcaFeatures                                                |
+-----------------------------------------------------------+
|[-156066.9999695493,181.9376571225013,12.618304586993649]  |
|[-220132.9999773746,135.2977242202464,11.697932158870637]  |
|[-250959.99997703813,140.66426868649597,11.269604456042869]|
|[-457039.99998047453,118.28340615864282,21.58643736964838] |
|[-282890.9999760614,144.01111681365217,19.639785493785286] |
|[-216999.99998004534,119.81952102725646,9.197255301752925] |
|[-545106.9999820379,107.31807697516126,20.978527661052837] |
|[-256932.99998395916,97.43991130277877,6.9474325850838055] |
|[-249132.99997462067,152.0528563593076,10.557161386762989] |
|[-189799.9999823239,102.80350444140416,17.485421375238687] |
|[-181732.99998730203,77.10903224443321,17.099317900674194] |
|[-202999.9999809192,115.75897125310874,10.382200366675361] |
|[-213239.99998251288,104.71808171761714,13.476058647273033]|
|[-28202

Scalo gli attributi numerici con il codice utilizzato nella correlation

In [56]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import MinMaxScaler

scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")

# Compute summary statistics and generate MinMaxScalerModel
scalerModel = scaler.fit(df)

# rescale each feature to range [min, max].
scaledData = scalerModel.transform(df)
print("Features scaled to range: [%f, %f]" % (scaler.getMin(), scaler.getMax()))
scaledData.select("features", "scaledFeatures").show()

Features scaled to range: [0.000000, 1.000000]
+--------------------+--------------------+
|            features|      scaledFeatures|
+--------------------+--------------------+
|[156067.0,0.602,-...|[0.02718519496877...|
|[220133.0,0.77,-7...|[0.03858915312091...|
|[250960.0,0.212,-...|[0.04407646024100...|
|[457040.0,0.362,-...|[0.08075937623378...|
|[282891.0,0.343,-...|[0.04976028298249...|
|[217000.0,0.893,-...|[0.03803146884390...|
|[545107.0,0.351,-...|[0.09643559111507...|
|[256933.0,0.422,-...|[0.04513967383390...|
|[249133.0,0.627,-...|[0.04375124824800...|
|[189800.0,0.554,-...|[0.03318977962125...|
|[181733.0,0.189,-...|[0.03175382715952...|
|[203000.0,0.46,-6...|[0.03553942292048...|
|[213240.0,0.764,-...|[0.03736217651018...|
|[282027.0,0.413,-...|[0.04960648814836...|
|[161213.0,0.829,-...|[0.02810119984891...|
|[237760.0,0.632,-...|[0.04172681694178...|
|[244253.0,0.442,-...|[0.04288259224040...|
|[207673.0,0.582,-...|[0.03637123224906...|
|[200667.0,0.881,-...|[0.0351

In [57]:
from pyspark.sql import functions as f
names = {x + "_scaled": x for x in columns_to_scale}
scaledData = enriched_df.select([f.col(c).alias(names[c]) for c in names.keys()])

In [58]:
df.toPandas()

,acousticness,danceability,duration_ms,genres,instrumentalness,key,liveness,loudness,mode,release_date,speechiness,tempo,time_signature,valence,features
0,0.658000,0.602,156067,"[classic czech pop, czech folk, czech rock]",0.000000,0,0.0972,-6.667,1,1980-01-01,0.4040,182.229,3,0.650,"[156067.0, 0.602, -6.667, 0.404, 0.658, 0.0, 0..."
1,0.543000,0.770,220133,"[afrobeat, afropop, world]",0.000796,1,0.0684,-7.306,1,1976-01-01,0.1720,135.573,4,0.898,"[220133.0, 0.77, -7.306, 0.172, 0.543, 0.00079..."
2,0.000048,0.212,250960,"[alternative metal, gothenburg metal, melodic ...",0.918000,0,0.3240,-6.690,0,1996-02-20,0.1400,140.917,4,0.231,"[250960.0, 0.212, -6.69, 0.14, 4.8e-05, 0.918,..."
3,0.144000,0.362,457040,"[corrosion, dark wave, ethereal wave, gothabil...",0.827000,11,0.1170,-17.744,0,1990-01-01,0.0398,118.853,4,0.257,"[457040.0, 0.362, -17.744, 0.0398, 0.144, 0.82..."
4,0.957000,0.343,282891,"[brazilian rock, mpb, pop rock brasileiro, roc...",0.000249,1,0.6610,-14.937,0,2017-09-22,0.0384,144.533,4,0.101,"[282891.0, 0.343, -14.937, 0.0384, 0.957, 0.00..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583358,0.158000,0.775,198549,[],0.000000,1,0.1000,-8.266,1,2019-04-30,0.1920,88.022,4,0.866,"[198549.0, 0.775, -8.266, 0.192, 0.158, 0.0, 0..."
583359,0.757000,0.544,213787,[hungarian pop],0.000005,10,0.2020,-3.735,1,2012-11-12,0.0967,79.836,4,0.620,"[213787.0, 0.544, -3.735, 0.0967, 0.757, 4.79e..."
583360,0.001150,0.580,279786,"[chilean indie, chilean rock, latin alternativ...",0.024200,7,0.6220,-9.005,1,2001-10-08,0.0454,89.983,4,0.671,"[279786.0, 0.58, -9.005, 0.0454, 0.00115, 0.02..."
583361,0.000092,0.621,421907,"[alternative rock, big beat, britpop, dance ro...",0.579000,8,0.8890,-14.754,1,1991-09-23,0.0514,91.147,4,0.885,"[421907.0, 0.621, -14.754, 0.0514, 9.23e-05, 0..."


# K-Means

In [59]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Trains a k-means model.
kmeans = KMeans().setK(10).setSeed(1)
model = kmeans.fit(df)

# Make predictions
predictions = model.transform(df)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Silhouette with squared euclidean distance = 0.6838578521487513
Cluster Centers: 
[ 2.32579550e+05  5.75756912e-01 -9.02219037e+00  7.22511317e-02
  3.63748774e-01  6.84041465e-02  2.00077628e-01  5.50122723e-01
  1.20248540e+02  3.91493545e+00]
[ 9.10610063e+05  3.98673249e-01 -1.50978575e+01  1.87547957e-01
  6.77114856e-01  3.91328392e-01  2.54966780e-01  3.22727539e-01
  1.07627835e+02  3.79669261e+00]
[ 9.79313173e+04  5.77987071e-01 -1.31676307e+01  3.20900199e-01
  5.80684706e-01  1.68166639e-01  2.78709140e-01  5.82318977e-01
  1.12835493e+02  3.73121091e+00]
[ 2.96809112e+05  5.50017801e-01 -9.65229965e+00  7.14611272e-02
  3.75107564e-01  9.37029966e-02  2.08234638e-01  5.00885526e-01
  1.19299125e+02  3.90975346e+00]
[ 5.75856466e+05  4.36007367e-01 -1.38569856e+01  1.09168551e-01
  5.49804366e-01  3.64776463e-01  2.56890822e-01  3.54052943e-01
  1.13347252e+02  3.83272149e+00]
[ 3.90204979e+06  4.46275362e-01 -1.06648623e+01  2.05805072e-01
  4.61046333e-01  2.77218294e-01 